In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [2]:
basedir = Path().resolve().parent
datadir = basedir / 'data'

In [18]:
train = pd.read_csv(datadir / 'train.csv')
train["RC"] = train["R"].astype(str) + '_' + train["C"].astype(str)
train.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure,RC
0,1,1,20,50,0.000000,0.083334,0,5.837492,20_50
1,2,1,20,50,0.033652,18.383041,0,5.907794,20_50
2,3,1,20,50,0.067514,22.509278,0,7.876254,20_50
3,4,1,20,50,0.101542,22.808822,0,11.742872,20_50
4,5,1,20,50,0.135756,25.355850,0,12.234987,20_50


In [19]:
RC_types = train.RC.unique()
train["kfold"] = -1

SEED = 42
N_SPLITS = 5

for RC_type in RC_types:
    breath_ids = train[train["RC"] == RC_type].breath_id.unique()
    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    
    for fold, (_, test_idx) in enumerate(kf.split(breath_ids)):
        valid_breath_ids = breath_ids[test_idx]
        train.loc[train["breath_id"].isin(valid_breath_ids), "kfold"] = fold

In [24]:
train.to_csv(datadir / f"train_RC_kfold{N_SPLITS}_seed{SEED}.csv")